# Direct Field Examination

Let's examine the actual field names to understand the pattern.

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from jcc2_data_processor import create_processor
import pandas as pd

In [ ]:
# Load data
data_file = '/home/nathanjgaul/Downloads/DCDC UQ Combined_2025-07-30_pii_scrubbed.csv'
processor = create_processor(data_file)
df = processor.load_data()
print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

In [ ]:
# Look at ALL fields in mop_1_1_1 section
print("ALL fields in mop_1_1_1:")
print("=" * 80)

if 'mop_1_1_1' in processor.sections:
    fields = sorted(processor.sections['mop_1_1_1'])
    for i, field in enumerate(fields):
        print(f"{i+1:3d}. {field}")
        # Check if it has data
        if field in df.columns:
            non_null = df[field].notna().sum()
            if non_null > 0:
                print(f"     -> Has {non_null} non-null values")
                # Show sample values
                sample = df[field].dropna().head(3).tolist()
                print(f"     -> Sample: {sample}")

In [ ]:
# Search for specific app fields
print("\nSearching for fields containing specific apps:")
print("=" * 80)

apps_to_search = ['threathub', 'a2it', 'cad', 'madss']

for app in apps_to_search:
    print(f"\nFields containing '{app}':")
    app_fields = [col for col in df.columns if app in col.lower()]
    
    if app_fields:
        for field in app_fields[:10]:  # First 10
            print(f"  - {field}")
            # Check section
            parts = field.split('.')
            if len(parts) > 1:
                print(f"    Section: {parts[0]}")
    else:
        print(f"  No fields found containing '{app}'")

In [ ]:
# Let's look for effectiveness/suitability fields specifically
print("\nEffectiveness/Suitability fields pattern analysis:")
print("=" * 80)

# Get all rating fields
rating_fields = [col for col in df.columns if any(x in col.lower() for x in ['effectiveness', 'suitability'])]

# Group by section
sections_with_ratings = {}
for field in rating_fields:
    parts = field.split('.')
    if len(parts) >= 2:
        section = parts[0]
        if section not in sections_with_ratings:
            sections_with_ratings[section] = []
        sections_with_ratings[section].append(field)

# Show fields for our target sections
target_sections = ['mop_1_1_1', 'mos_1_1_2', 'reporting_and_data_export']

for section in target_sections:
    if section in sections_with_ratings:
        print(f"\n{section}:")
        fields = sections_with_ratings[section]
        print(f"  Total rating fields: {len(fields)}")
        
        # Show examples
        for field in fields[:5]:
            print(f"  - {field}")
            # Extract the part after section name
            field_part = field.replace(f"{section}.", "")
            print(f"    Field part: '{field_part}'")
            
            # Check for app names
            for app in ['threathub', 'a2it', 'cad', 'madss', 'sigact', 'unity']:
                if app in field_part:
                    print(f"    Contains app: {app}")

In [ ]:
# Let's look at the actual pattern by examining field endings
print("\nAnalyzing field name patterns:")
print("=" * 80)

# Get effectiveness fields from mop_1_1_1
mop_fields = [f for f in df.columns if f.startswith('mop_1_1_1') and 'effectiveness' in f]

print(f"\nMOP 1.1.1 effectiveness fields ({len(mop_fields)} total):")

# Extract the endings
endings = []
for field in mop_fields:
    # Remove the section prefix
    without_section = field.replace('mop_1_1_1.', '')
    print(f"\n  Field: {field}")
    print(f"  Without section: {without_section}")
    
    # Try to identify the pattern
    parts = without_section.split('_')
    if len(parts) > 0:
        last_part = parts[-1]
        print(f"  Last part: {last_part}")
        
        # Check if last part is an app
        apps = ['threathub', 'a2it', 'cad', 'codex', 'crucible', 'dispatch', 
                'jcc2cyberops', 'jcc2readiness', 'madss', 'rally', 'redmap',
                'sigact', 'triage', 'unity']
        
        if last_part in apps:
            print(f"  -> FOUND APP: {last_part}")
        elif 'overall' in without_section:
            print(f"  -> OVERALL FIELD")
        else:
            print(f"  -> UNKNOWN PATTERN")